In [1]:
import ifm3dpy
from ifm3dpy import * 
import json
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import cv2
import open3d as o3d
import pandas as pd
import numpy as np
import copy
import time
import tensorflow as tf
import pickle
import keyboard
import segmentation_models as sm
import asyncio
from PIL import Image
from scipy.interpolate import RectBivariateSpline
print(o3d)
print(o3d.core.cuda.is_available())

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Segmentation Models: using `keras` framework.
<module 'open3d' from 'C:\\Users\\WSADMIN\\anaconda3\\lib\\site-packages\\open3d\\__init__.py'>
False


In [2]:
import numpy as np
import segmentation_models as sm
import cv2
from PIL import Image
sm.set_framework('tf.keras')

class HandlingUnitExtractor():
    def __init__(self):
        model_file = 'best_model_handling_unit.h5'
        self.model_img_size = (512, 512)
        self.model = sm.Unet('efficientnetb3', classes=1, activation='sigmoid')
        self.model.load_weights(model_file) 
        self.preprocessor = sm.get_preprocessing('efficientnetb3')
        
    def extract(self, img):
        h, w = img.shape[:2]
        image = cv2.resize(img, self.model_img_size, interpolation = cv2.INTER_AREA)
        image = self.preprocessor(image)
        
        #t = time.time()
        #mask = self.model.predict(np.expand_dims(image, axis=0)).round().squeeze()
        mask = self.model(np.expand_dims(image, axis=0)).numpy()
        #print(mask)
        mask = mask.round().squeeze()
        #print(time.time() -t)
        mask = cv2.resize(mask, (w, h), interpolation = cv2.INTER_AREA)
        
        mask = self.isolate_largest_component(mask)
        
        return mask.astype(np.uint8)

    def isolate_largest_component(self, img):
        result = np.zeros((img.shape))
        labels, stats = cv2.connectedComponentsWithStats(img.astype(np.uint8), connectivity=8)[1:3]                   
        
        largest_label = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
        result[labels == largest_label] = 255 

        return result

In [3]:
def turnoffO3R(o3r):
    config = o3r.get()
    config['ports']['port0']['state'] = 'IDLE'
    config['ports']['port2']['state'] = 'IDLE'
    o3r.set(config)
    
def turnonO3R(o3r):
    config = o3r.get()
    config['ports']['port0']['state'] = 'RUN'
    config['ports']['port2']['state'] = 'RUN'
    o3r.set(config)

# def RectificationalizationColor(image, intrinsics):    

#     fx, fy, mx, my, alpha, k1, k2, k5, k3, k4, *_ = intrinsics

#     dmap = image
    
#     KKNoAlpha = np.array([[fx, 0.0, mx],
#                           [0,  fy,  my],
#                           [0,  0,   1]])

#     # Generate the pixel coordinates
#     px,py = np.meshgrid(np.arange(0, dmap.shape[1]),
#                         np.arange(0, dmap.shape[0]))

#     # Homogeneous coordinates
#     coords = np.hstack([px.reshape(-1,1) + 0.5, py.reshape(-1,1) + 0.5, np.ones([px.size, 1])]).T

#     # Transfer into a normalized coordinate frame
#     # (f=1, origin is the center, Dim: -1..1)
#     coords_norm = np.linalg.solve(KKNoAlpha, coords)
    
#     # Apply distortion
#     R2 = np.square(coords_norm[0]) + np.square(coords_norm[1])
#     R4 = np.square(R2)
#     R6 = np.power(R2,3)
#     radial_dist = 1 + k1*R2 + k2*R4 + k5*R6

#     a1 = 2*coords_norm[0]*coords_norm[1]
#     a2 = R2 + 2*np.square(coords_norm[0])
#     a3 = R2 + 2*np.square(coords_norm[1])
#     tangential_dist = np.vstack([k3*a1 + k4*a2, k3*a3 + k4*a1])

#     coord_dist = np.vstack([np.ones([2,1])*radial_dist*coords_norm[0:2] + tangential_dist, np.ones([1,px.size])])

#     # Transformer --> Pixel
#     KK = KKNoAlpha
#     KK[0,1] = fx*alpha
#     #KK[0,1] = alpha
#     coord_dist = KK.dot(coord_dist)
    
    
#     fr = RectBivariateSpline(np.arange(0, dmap.shape[0]),
#                         np.arange(0, dmap.shape[1]),
#                         dmap[:,:,0])
#     fg = RectBivariateSpline(np.arange(0, dmap.shape[0]),
#                         np.arange(0, dmap.shape[1]),
#                         dmap[:,:,1])
#     fb = RectBivariateSpline(np.arange(0, dmap.shape[0]),
#                         np.arange(0, dmap.shape[1]),
#                         dmap[:,:,2])

#     rectifiedr = fr(coord_dist[1]-0.5, coord_dist[0]-0.5, grid=False) /255
#     rectifiedg = fg(coord_dist[1]-0.5, coord_dist[0]-0.5, grid=False) /255
#     rectifiedb =  fb(coord_dist[1]-0.5, coord_dist[0]-0.5, grid=False)/255

    


#     amp_rectifiedr = rectifiedr.reshape(dmap.shape[:2])
#     amp_rectifiedg = rectifiedg.reshape(dmap.shape[:2])
#     amp_rectifiedb = rectifiedb.reshape(dmap.shape[:2])

    
#     img = np.stack([amp_rectifiedr,amp_rectifiedg,amp_rectifiedb], axis=2)
#     #img = img*255
#     #print(img)
#     #print(img.shape)
#     #plt.imshow(img)
#     return img
    
# def Rectificationalization(image, intrinsics):
    
#     amp = image
#     # Unpack intrinsics values
#     fx, fy, mx, my, alpha, k1, k2, k5, k3, k4, *_ = intrinsics

#     # Create the camera matrix
#     # If the current application is using the 23k imager, fx/fy/mx/my must be divided by two.
#     # If the current application is using the full resolution imager, fx/fy/mx/my can be used 'as-is'
    
    
#     KKNoAlpha = np.array([[fx, 0.0, mx],
#                               [0,  fy,  my],
#                               [0,  0,   1]])

#     # Generate the pixel coordinates
#     px,py = np.meshgrid(np.arange(0, amp.shape[1]),
#                         np.arange(0, amp.shape[0]))

#     # Homogeneous coordinates
#     coords = np.hstack([px.reshape(-1,1) + 0.5, py.reshape(-1,1) + 0.5, np.ones([px.size, 1])]).T

#     # Transfer into a normalized coordinate frame
#     # (f=1, origin is the center, Dim: -1..1)
#     coords_norm = np.linalg.solve(KKNoAlpha, coords)
    
#     # Apply distortion
#     R2 = np.square(coords_norm[0]) + np.square(coords_norm[1])
#     R4 = np.square(R2)
#     R6 = np.power(R2,3)
#     radial_dist = 1 + k1*R2 + k2*R4 + k5*R6

#     a1 = 2*coords_norm[0]*coords_norm[1]
#     a2 = R2 + 2*np.square(coords_norm[0])
#     a3 = R2 + 2*np.square(coords_norm[1])
#     tangential_dist = np.vstack([k3*a1 + k4*a2, k3*a3 + k4*a1])

#     coord_dist = np.vstack([np.ones([2,1])*radial_dist*coords_norm[0:2] + tangential_dist,
#                        np.ones([1,px.size])])

#     # Transformer --> Pixel
#     KK = KKNoAlpha
#     KK[0,1] = fx*alpha
#     #KK[0,1] = alpha
#     coord_dist = KK.dot(coord_dist)
    
#     f = RectBivariateSpline(np.arange(0, amp.shape[0]),
#                         np.arange(0, amp.shape[1]),
#                         amp)
#     rectified = f(coord_dist[1]-0.5, coord_dist[0]-0.5, grid=False)
#     amp_rectified = rectified.reshape(amp.shape)
#     return amp_rectified

def visualize(**images):
    n = len(images)
    plt.figure(figsize=(16, 8))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')) + f' {image.shape} {image.shape[1] / image.shape[0]:.2f}'.title())
        plt.imshow(image)
    plt.show()

def preprocess_point_cloud(pcd, voxel_size):
    #print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)
    #pcd_down = pcd.to_legacy()
    radius_normal = voxel_size * 2
    #print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    #print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.5
    #print(":: Apply fast global registration with distance threshold %.3f" \ % distance_threshold)
    result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    #print(":: RANSAC registration on downsampled point clouds.")
    #print("   Since the downsampling voxel size is %.3f," % voxel_size)
    #print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    #distance_threshold = voxel_size * 0.4
    distance_threshold = voxel_size * 0.4
    #print(":: Point-to-plane ICP registration is applied on original point")
    #print("   clouds to refine the alignment. This time we use a strict")
    #print("   distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
    return result

def refine_registrationP2P(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    #print(":: Point-to-plane ICP registration is applied on original point")
    #print("   clouds to refine the alignment. This time we use a strict")
    #print("   distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
    return result


def undistort(img, config):
    
    mtx, newcameramtx, roi, dist = config
    dst = cv2.undistort(img, mtx, dist, newcameramtx)
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    return dst

def loadpickle(filename = 'bag'):
    with open(filename, "rb") as fp:
            frames = pickle.load(fp)
    return frames


    

            

# rgb = cv2.imdecode(frames[0][1], cv2.IMREAD_UNCHANGED) 
# rgb = undistort(rgb)
# rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)  
# rgb = rgb[:, 95:-95]
# rgb = cv2.resize(rgb, (224,172), interpolation = cv2.INTER_AREA)
# plt.imshow(rgb)


    
    
def load_point_clouds(frames = None, numframes=200, picklefile= 'bag2', config=loadpickle('rgb_camera_config')): 
    pcds = []
    maskpcds = []
    hue = HandlingUnitExtractor()
    for frame in frames:    
        rgb = frame[1]
        xyz = frame[0]  
        
        rgb = cv2.imdecode(rgb, cv2.IMREAD_UNCHANGED)
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        
        rgb = undistort(rgb, config)
        
        padup = np.zeros((100,rgb.shape[1],3))
        paddown = np.zeros((100,rgb.shape[1],3)) 
        rgb = np.concatenate([padup, rgb, paddown], axis=0)
        
        #    pic = pic[:, 119:-119]
        # pic = cv2.resize(pic, (224,172), interpolation = cv2.INTER_AREA)
        # print(pic.shape)
        # pic = pic/255
        
#         rgb = cv2.imdecode(rgb, cv2.IMREAD_UNCHANGED)
#         rgb = undistort(rgb)         
#         rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)  
#         rgb = rgb[:, 95:-95]
        
        #mask = hue.extract(rgb)
        
        #rgb = rgb[1:-1, 70:-70]
        up = 110
        down = -126
        left = 144
        right = -166
        rgb = rgb[up : down, left : right] 
        #rgb = rgb[110:-126, 119:-119]
        
        rgb = cv2.resize(rgb, (224,172), interpolation = cv2.INTER_AREA)
        
        #mask = mask[:, 119:-119]        
        #mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB) 
        #mask[:,:,1] = 0   
        #mask = cv2.resize(mask, (224,172), interpolation = cv2.INTER_AREA)
       
        #rgb = undistort(rgb)
        
        rgb = rgb.reshape((rgb.shape[0] * rgb.shape[1], 3))
        rgb = rgb/255
        
        #mask = mask.reshape((mask.shape[0] * mask.shape[1], 3))
        #mask = mask/255
        
        xyz = xyz.reshape((xyz.shape[0] * xyz.shape[1], 3))     
        
        
        pcl = o3d.geometry.PointCloud()        
        pcl.points = o3d.utility.Vector3dVector(xyz)    
        pcl.colors = o3d.utility.Vector3dVector(rgb)         
        pcds.append(pcl)    
        
        # maskpcl = o3d.geometry.PointCloud()        
        # maskpcl.points = o3d.utility.Vector3dVector(xyz)    
        # maskpcl.colors = o3d.utility.Vector3dVector(mask)         
        # maskpcds.append(maskpcl)    
        
    
    return  pcds, maskpcds

def pairwise_registration(source, target):    
    voxel_sizes = o3d.utility.DoubleVector([0.1, 0.05, 0.025])

    # List of Convergence-Criteria for Multi-Scale ICP:
    criteria_list = [
        o3d.t.pipelines.registration.ICPConvergenceCriteria(relative_fitness=0.0001, relative_rmse=0.0001, max_iteration=20),
        o3d.t.pipelines.registration.ICPConvergenceCriteria(0.00001, 0.00001, 15),
        o3d.t.pipelines.registration.ICPConvergenceCriteria(0.000001, 0.000001, 10)
    ]

    # `max_correspondence_distances` for Multi-Scale ICP (o3d.utility.DoubleVector):
    max_correspondence_distances = o3d.utility.DoubleVector([0.3, 0.14, 0.07])

    # Initial alignment or source to target transform.
    init_source_to_target = o3d.core.Tensor.eye(4, o3d.core.Dtype.Float32)

    mu, sigma = 0, .1  # mean and standard deviation
    # Select the `Estimation Method`, and `Robust Kernel` (for outlier-rejection).
    #estimation = o3d.t.pipelines.registration.TransformationEstimationPointToPoint()
    #estimation = o3d.t.pipelines.registration.TransformationEstimationPointToPlane(o3d.t.pipelines.registration.robust_kernel.RobustKernel(
    #        o3d.t.pipelines.registration.robust_kernel.RobustKernelMethod.TukeyLoss, sigma))
    #estimation = o3d.t.pipelines.registration.TransformationEstimationForGeneralizedICP()
    
    estimation = o3d.t.pipelines.registration.TransformationEstimationForColoredICP()#o3d.t.pipelines.registration.robust_kernel.RobustKernel(
    #        o3d.t.pipelines.registration.robust_kernel.RobustKernelMethod.TukeyLoss, sigma))

    #Save iteration wise `fitness`, `inlier_rmse`, etc. to analyse and tune result.
    save_loss_log = True
    
    s = time.time()
    

    registration_ms_icp = o3d.t.pipelines.registration.multi_scale_icp(source, target,
                                           voxel_sizes, criteria_list,
                                           max_correspondence_distances, estimation_method = estimation)
    
    transformation_icp = registration_ms_icp.transformation
    information_icp = o3d.t.pipelines.registration.get_information_matrix(source, target, max_correspondence_distances[2], registration_ms_icp.transformation)
    
    ms_icp_time = time.time() - s
    #print("Time taken by Multi-Scale ICP: ", ms_icp_time)
    #print("Inlier Fitness: ", registration_ms_icp.fitness)
    #print("Inlier RMSE: ", registration_ms_icp.inlier_rmse)
    
    return transformation_icp, information_icp


def full_registration(pcds):
    pose_graph = o3d.pipelines.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)

    
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(pcds[source_id], pcds[target_id])
            
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp.numpy(), odometry)
                pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(np.linalg.inv(odometry)))
                pose_graph.edges.append(o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp.numpy(),
                                                             information_icp.numpy(),
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp.numpy(),
                                                             information_icp.numpy(),
                                                             uncertain=True))
    return pose_graph



def plot_rmse(registration_result):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20, 5))
    axes.set_title("Inlier RMSE vs Iteration")
    axes.plot(registration_result.loss_log["index"].numpy(),
              registration_result.loss_log["inlier_rmse"].numpy())
    
def plot_fitness(registration_result):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20, 5))
    axes.set_title("Inlier RMSE vs Iteration")
    axes.plot(registration_result.loss_log["index"].numpy(),
              registration_result.loss_log["fitness"].numpy())


def plot_scale_wise_rmse(registration_result):
    scales = registration_result.loss_log["scale"].numpy()
    iterations = registration_result.loss_log["iteration"].numpy()

    num_scales = scales[-1][0] + 1

    fig, axes = plt.subplots(nrows=1, ncols=num_scales, figsize=(20, 5))

    masks = {}
    for scale in range(0, num_scales):
        masks[scale] = registration_result.loss_log["scale"] == scale

        rmse = registration_result.loss_log["inlier_rmse"][masks[scale]].numpy()
        iteration = registration_result.loss_log["iteration"][
            masks[scale]].numpy()

        title_prefix = "Scale Index: " + str(scale)
        axes[scale].set_title(title_prefix + " Inlier RMSE vs Iteration")
        axes[scale].plot(iteration, rmse)

def draw_registration_result(source, target, transformation):
    source_temp = source.clone()
    target_temp = target.clone()

    source_temp.transform(transformation)

    # This is patched version for tutorial rendering.
    # Use `draw` function for you application.
    o3d.visualization.draw_geometries(
        [source_temp.to_legacy(),
         target_temp.to_legacy()] )
    
def VisualizePointCloudMovie(pcds):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcds[0])

    i = 1
    while not keyboard.is_pressed('esc'):    
        if keyboard.is_pressed('right') and i < len(pcds)-1:
            vis.clear_geometries()
            vis.add_geometry(pcds[i], False)
            #vis.update_geometry(pcds[0].transform(transforms[i]), False)
            i = i + 1
        if keyboard.is_pressed('left') and i > 0:
            vis.clear_geometries()
            vis.add_geometry(pcds[i], False)
            #vis.update_geometry(pcds[0].transform(transforms[i]), False)
            i = i - 1
            s = time.time()
    
        vis.poll_events()
        vis.update_renderer()  
    vis.destroy_window()
        
def collectframes(picklefile = 'realbag', numframes=300, savebag = False):    
    frames = []
    for i in range(numframes):
        success3d = fg3d.wait_for_frame(im3d)
        success2d = fg2d.wait_for_frame(im2d)
        if not success2d:
            print('2d failed!')
        if not success3d:
            print('3d failed!')
        frames.append((im3d.xyz_image() , im2d.jpeg_image(), im3d.distance_image()))   
        #time.sleep(.1)
    
    if savebag:
        with open(picklefile, "wb") as fp:
            pickle.dump(frames, fp)
    
    return frames

In [4]:
o3r = O3RCamera()
config = o3r.get()
config['ports']['port0']['acquisition']['framerate'] = 20
config['ports']['port2']['acquisition']['framerate'] = 20
config['ports']['port0']['state'] = 'RUN'
config['ports']['port2']['state'] = 'RUN'
#config['ports']['port0']['processing']['diParam']['maxDistNoise'] = 0.02
fg3d = FrameGrabber(o3r, 10, 50010)
im3d = ImageBuffer()
fg2d = FrameGrabber(o3r, 10, 50012)
im2d = ImageBuffer()

o3r.set(config)
print(json.dumps(config,indent=4))


{
    "device": {
        "clock": {
            "currentTime": 1581090794560201440
        },
        "diagnostic": {
            "temperatures": [],
            "upTime": 150000000000
        },
        "info": {
            "device": "0301",
            "deviceTreeBinaryBlob": "tegra186-quill-p3310-1000-c03-00-base.dtb",
            "features": {},
            "name": "",
            "partNumber": "M03975",
            "productionState": "AA",
            "serialNumber": "00020142B090",
            "vendor": "0001"
        },
        "network": {
            "authorized_keys": "",
            "ipAddressConfig": 0,
            "macEth0": "48:B0:2D:3A:68:01",
            "macEth1": "00:02:01:42:B0:90",
            "networkSpeed": 1000,
            "staticIPv4Address": "192.168.0.69",
            "staticIPv4Gateway": "192.168.0.201",
            "staticIPv4SubNetMask": "255.255.255.0",
            "useDHCP": false
        },
        "state": {
            "errorMessage": "",
          

In [ ]:
#time.sleep(15)
s = time.time()
#frames = loadpickle('bag4')
#frames = loadpickle('realbag')
frames = collectframes('realbag', 300, True)
print('DONE Collecting ' + str(time.time() - s))

s = time.time()
config = loadpickle('rgb_camera_config')

pcds, maskpcds = load_point_clouds(frames, config=config)
print(time.time() - s) 
#o3d.visualization.draw_geometries(pcds)

DONE Collecting 17.048710107803345


In [ ]:
VisualizePointCloudMovie(pcds)

In [ ]:


def moveimage(pcd, direction, left, right, up, down):    
    rgb = frames[i][1]
    rgb = cv2.imdecode(rgb, cv2.IMREAD_UNCHANGED)
    rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB) 
    rgb = undistort(rgb,config)
    
    padup = np.zeros((100,rgb.shape[1],3))
    paddown = np.zeros((100,rgb.shape[1],3))
    
    rgb = np.concatenate([padup, rgb, paddown], axis=0)      
    
    if direction == 'left':
        left = left - 1
        right = right - 1
    elif direction == 'right':
        left = left + 1
        right = right + 1  
    elif direction == 'up':
        up = up - 1
        down = down - 1   
    elif direction == 'down':
        up = up + 1
        down = down + 1
    elif direction == 'a':
        left = left - 1
        right = right + 1
    elif direction == 's':
        left = left + 1
        right = right - 1
    elif direction == 'q':
        up = up - 1
        down = down + 1
    elif direction == 'w':
        up = up + 1
        down = down - 1       
    
    
    rgb = rgb[up : down, left : right] 
    rgb = cv2.resize(rgb, (224,172), interpolation = cv2.INTER_AREA)
    rgb = rgb.reshape((rgb.shape[0] * rgb.shape[1], 3))
    rgb = rgb/255 
    pcd.colors = o3d.utility.Vector3dVector(rgb)
    
    return pcd, left, right, up, down

up = 64
down = -90
left = 83
right = -109
s = time.time()
i = 1
vis = o3d.visualization.Visualizer()
vis.create_window()
pcdscal = [copy.deepcopy(x) for x in pcds]

vis.add_geometry(copy.deepcopy(pcdscal[i]))
config = loadpickle('rgb_camera_config')

while not keyboard.is_pressed('esc'):  
   
    if keyboard.is_pressed('up'): 
        pcdscal[i], left, right, up, down = moveimage(pcdscal[i], 'up', left, right, up, down)
        vis.clear_geometries()
        vis.add_geometry(pcdscal[i], False)
    if keyboard.is_pressed('down'):
        pcdscal[i], left, right, up, down = moveimage(pcdscal[i], 'down', left, right, up, down)
        vis.clear_geometries()
        vis.add_geometry(pcdscal[i], False)
    if keyboard.is_pressed('left'): 
        pcdscal[i], left, right, up, down = moveimage(pcdscal[i], 'left', left, right, up, down)
        vis.clear_geometries()
        vis.add_geometry(pcdscal[i], False)
    if keyboard.is_pressed('right'): 
        pcdscal[i], left, right, up, down = moveimage(pcdscal[i], 'right', left, right, up, down)
        vis.clear_geometries()
        vis.add_geometry(pcdscal[i], False)
    
    if keyboard.is_pressed('a'): 
        pcdscal[i], left, right, up, down = moveimage(pcdscal[i], 'a', left, right, up, down)
        vis.clear_geometries()
        vis.add_geometry(pcdscal[i], False)
        
    if keyboard.is_pressed('s'): 
        pcdscal[i], left, right, up, down = moveimage(pcdscal[i], 's', left, right, up, down)
        vis.clear_geometries()
        vis.add_geometry(pcdscal[i], False)
        
    if keyboard.is_pressed('q'): 
        pcdscal[i], left, right, up, down = moveimage(pcdscal[i], 'q', left, right, up, down)
        vis.clear_geometries()
        vis.add_geometry(pcdscal[i], False)
        
    if keyboard.is_pressed('w'): 
        pcdscal[i], left, right, up, down = moveimage(pcdscal[i], 'w', left, right, up, down)
        vis.clear_geometries()
        vis.add_geometry(pcdscal[i], False)
        
    if keyboard.is_pressed('space') and i < len(pcdscal)-1:
            pcdscal[i], left, right, up, down = moveimage(pcdscal[i], '', left, right, up, down)
            vis.clear_geometries()            
            vis.add_geometry(pcdscal[i], False)
            i = i + 1
    if keyboard.is_pressed('backspace') and i > 0:
            pcdscal[i], left, right, up, down = moveimage(pcdscal[i], '', left, right, up, down)
            vis.clear_geometries()
            vis.add_geometry(pcdscal[i], False)
            i = i - 1
            s = time.time()
            
    moveimage(pcdscal[i], '', left, right, up, down)
    
    vis.poll_events()
    vis.update_renderer()
print(up)
print(down)
print(left)
print(right)
vis.destroy_window()

In [ ]:
o3d.visualization.draw_geometries([pcds[0]])

In [ ]:
%matplotlib qt

In [ ]:
newpcd = []
cnt = 1
checkevery = 5000
for i in range(0,400):     
    pcd = pcds[i].voxel_down_sample(voxel_size=0.01)
    
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.04, ransac_n=3, num_iterations=1000)
    [a, b, c, d] = plane_model
    
    if cnt % checkevery == 0:
        o3d.visualization.draw_geometries([pcd])
        
    inlier_cloud = pcd.select_by_index(inliers)
    inlier_cloud.paint_uniform_color([1.0, 0, 0])
    pcd = pcd.select_by_index(inliers, invert=True)
    
    if cnt % checkevery == 0:
        o3d.visualization.draw_geometries([pcd])
    
    
    pcd, o = pcd.remove_radius_outlier(15, .1, True)
    
    if cnt % checkevery == 0:
        o3d.visualization.draw_geometries([pcd])
    
    
    #with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd.cluster_dbscan(eps=0.13, min_points=50))  
       
    unique, counts = np.unique(labels[labels > -1], return_counts=True)    
    
    idxs = np.where(labels == unique[np.argmax(counts)])[0]
    
    cpcd = pcd.select_by_index(idxs)
    
    #cpcd, o = cpcd.remove_radius_outlier(15, .1, True)
        
    max_label = labels.max()
    
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    
    
    if cnt % checkevery == 0:    
        o3d.visualization.draw_geometries([pcd])
    
    if cnt % checkevery == 0:
        o3d.visualization.draw_geometries([cpcd])
    
    pcd = cpcd
    
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=30))
    pcd.orient_normals_towards_camera_location()
    newpcd.append(pcd)
    cnt = cnt + 1      
    

In [ ]:


#source = pcds[0]
#target = pcds[1]
source = copy.deepcopy(newpcd[0])
target = copy.deepcopy(newpcd[10])

source = source.paint_uniform_color([1.0, 0, 0])
target = target.paint_uniform_color([0, 0, 1.0])

o3d.visualization.draw_geometries([source, target])

voxel_sizes = o3d.utility.DoubleVector([0.15, 0.05, 0.025])

# List of Convergence-Criteria for Multi-Scale ICP:
criteria_list = [
    o3d.t.pipelines.registration.ICPConvergenceCriteria(relative_fitness=0.0001,
                                relative_rmse=0.0001,
                                max_iteration=20),
    o3d.t.pipelines.registration.ICPConvergenceCriteria(0.0001, 0.0001, 15),
    o3d.t.pipelines.registration.ICPConvergenceCriteria(0.00001, 0.00001, 10)
]

# `max_correspondence_distances` for Multi-Scale ICP (o3d.utility.DoubleVector):
max_correspondence_distances = o3d.utility.DoubleVector([0.4, 0.14, 0.05])

# Initial alignment or source to target transform.
init_source_to_target = o3d.core.Tensor.eye(4, o3d.core.Dtype.Float32)

# Select the `Estimation Method`, and `Robust Kernel` (for outlier-rejection).
#estimation = o3d.t.pipelines.registration.TransformationEstimationPointToPlane()
estimation = o3d.t.pipelines.registration.TransformationEstimationForColoredICP()
#estimation = o3d.t.pipelines.registration.TransformationEstimationForGeneralizedICP()
# mu, sigma = 0, 1 
# estimation = o3d.t.pipelines.registration.TransformationEstimationForColoredICP(o3d.t.pipelines.registration.robust_kernel.RobustKernel(
#             o3d.t.pipelines.registration.robust_kernel.RobustKernelMethod.TukeyLoss, sigma))

s = time.time()

source = o3d.t.geometry.PointCloud().from_legacy(source)
target = o3d.t.geometry.PointCloud().from_legacy(target)

registration_ms_icp = o3d.t.pipelines.registration.multi_scale_icp(source, target, voxel_sizes,
                                           criteria_list,
                                           max_correspondence_distances,
                                           estimation_method = estimation,
                                           save_loss_log=True)

ms_icp_time = time.time() - s
print("Time taken by Multi-Scale ICP: ", ms_icp_time)
print("Inlier Fitness: ", registration_ms_icp.fitness)
print("Inlier RMSE: ", registration_ms_icp.inlier_rmse)

draw_registration_result(source,target,registration_ms_icp.transformation)

plot_rmse(registration_ms_icp)
plot_scale_wise_rmse(registration_ms_icp)

In [ ]:
def multiscaleicp(s, t):
    voxel_sizes = o3d.utility.DoubleVector([0.1, 0.05, 0.025])
    criteria_list = [
        o3d.t.pipelines.registration.ICPConvergenceCriteria(relative_fitness=0.0001, relative_rmse=0.0001, max_iteration=20),
        o3d.t.pipelines.registration.ICPConvergenceCriteria(0.00001, 0.00001, 15),
        o3d.t.pipelines.registration.ICPConvergenceCriteria(0.000001, 0.000001, 10),
    ]

    max_correspondence_distances = o3d.utility.DoubleVector([0.3, 0.14, 0.07])


    # Select the `Estimation Method`, and `Robust Kernel` (for outlier-rejection).
    #estimation = o3d.t.pipelines.registration.TransformationEstimationPointToPlane()
    #estimation = o3d.t.pipelines.registration.TransformationEstimationForColoredICP()
    mu, sigma = 0, 1 
    estimation = o3d.t.pipelines.registration.TransformationEstimationForColoredICP(o3d.t.pipelines.registration.robust_kernel.RobustKernel(
                o3d.t.pipelines.registration.robust_kernel.RobustKernelMethod.TukeyLoss, sigma))
    
    registration_ms_icp = o3d.t.pipelines.registration.multi_scale_icp(s, t, voxel_sizes,
                                           criteria_list,
                                           max_correspondence_distances,
                                           estimation_method = estimation,
                                           save_loss_log=True)
    
    return registration_ms_icp


In [ ]:
pcds = [o3d.t.geometry.PointCloud().from_legacy(copy.deepcopy(i)) for i in newpcd]

tottime = time.time()
master =  pcds[0].clone()


masters = []      
source_cuda = 0
toofar = False
end = 250
i = 0
while i < end:    
    s = time.time() 
        
    target_cuda = pcds[i]
    
    registration_ms_icp = multiscaleicp(pcds[source_cuda], target_cuda)
    #registration_ms_icp = multiscaleicp(master, target_cuda)
    #registration_ms_icp = multiscaleicp(master, target_cuda) 

    ms_icp_time = time.time() - s
    scale = registration_ms_icp.loss_log['scale'].numpy()
    fitness = registration_ms_icp.loss_log['fitness'].numpy()
    rmse = registration_ms_icp.loss_log['inlier_rmse'].numpy()
    m = np.hstack([scale, fitness])
    m = m[m[:,0] == 2,:]
    m = m[0][1]
    e = np.hstack([scale, rmse])
    e = e[e[:,0] == 2,:]
    e = e[0][1]
    
    #print(e)
    if m > .95 or e < .022:# or m < .8:#and not toofar:  
        i = i + 1
        continue
        
    # if toofar:
    #     toofar = False
    
    print(str(source_cuda) + ' ' + str(i))
    print("Time taken by Multi-Scale ICP: ", ms_icp_time)
    print("Inlier Fitness: ", registration_ms_icp.fitness)
    print("Inlier RMSE: ", registration_ms_icp.inlier_rmse)
    
    if registration_ms_icp.fitness < .9:  
        print('************************************************** ' + str(source_cuda) + ' '+ str(i))
        #master = master.voxel_down_sample(voxel_size=0.025)
#         registration_ms_icp = multiscaleicp(master, target_cuda)
#         print("Inlier Fitness: ", registration_ms_icp.fitness)
#         print("Inlier RMSE: ", registration_ms_icp.inlier_rmse)
#         pcds[source_cuda].transform(registration_ms_icp.transformation)
#         target_cuda = o3d.t.geometry.PointCloud().from_legacy(target_cuda.to_legacy().paint_uniform_color([1.0, 0, 0]))
        
        # toofar = True
        # i = i-1
        # continue
    
    
    #transforms.append(registration_ms_icp.transformation)        
    master =  master.transform(registration_ms_icp.transformation) + target_cuda
    #o3d.visualization.draw_geometries([master.to_legacy()])       
    #masters.append(pcds[source_cuda].transform(registration_ms_icp.transformation).to_legacy())
    #masters.append(master.clone().to_legacy())
    source_cuda = i
    i = i + 1
    
        

In [ ]:
o3d.visualization.draw_geometries([target_cuda.to_legacy()])

In [ ]:
o3d.visualization.draw_geometries([master.to_legacy()])

In [ ]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(masters[0])

s = time.time()
i = 1
while not keyboard.is_pressed('esc'):    
    if time.time() - s > .5 and i < len(masters):
        vis.add_geometry(masters[i], False)
        #vis.update_geometry(pcds[0].transform(transforms[i]), False)
        i = i + 1
        s = time.time()
    
    vis.poll_events()
    vis.update_renderer()  
vis.destroy_window()

In [ ]:
pcds = [o3d.t.geometry.PointCloud().from_legacy(i) for i in newpcd]
tottime = time.time()
master = None
#hotpink =  o3d.t.geometry.PointCloud().from_legacy(maskpcds[0])


step = 30
#combinedpc =  o3d.geometry.PointCloud()
for i in range(0, 300, step):    
    s = time.time()
    print(str(i) + " " + str(i+step+1))
    graphpcs = pcds[i:i+step+1]
    #print(len(graphpcs))
    pose_graph = full_registration(graphpcs)
    #print(len(pose_graph.nodes))
    print("Optimizing PoseGraph ...")
    option = o3d.pipelines.registration.GlobalOptimizationOption(max_correspondence_distance=max_correspondence_distances[2],
                                                                 edge_prune_threshold=0.25,
                                                                 reference_node=step)
    
    o3d.pipelines.registration.global_optimization(pose_graph,
        o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
        o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
        option)
    
    
    
    # with o3d.utility.VerbosityContextManager(
    #         o3d.utility.VerbosityLevel.Debug) as cm:
    #     o3d.pipelines.registration.global_optimization(
    #         pose_graph,
    #         o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
    #         o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
    #         option)
    
    
    print("Transform points and display")
    
    master.transform(pose_graph.nodes[0].pose)
    for point_id in range(1,len(graphpcs)):        
        graphpcs[point_id].transform(pose_graph.nodes[point_id].pose)  
    
    
        master = master + graphpcs[point_id]
        #legacy.append(graphpcs[point_id].to_legacy())
    #o3d.visualization.draw_geometries([ x.to_legacy() for x in graphpcs ] )
    #o3d.visualization.draw_geometries([master.to_legacy()])
    
    
    
    # registration_ms_icp = o3d.t.pipelines.registration.multi_scale_icp(source_cuda, target_cuda,
    #                                        voxel_sizes, criteria_list,
    #                                        max_correspondence_distances, estimation_method = estimation)#, init_source_to_target = result_ransac.transformation)#, save_loss_log=False)#,
    #                                        #result_ransac.transformation, estimation,
    #                                        #save_loss_log)

    # source_cuda = source_cuda.cpu()
    # target_cuda = target_cuda.cpu()
    ms_icp_time = time.time() - s
    print("Time taken Pose Estimation for " + str(step) + " frames: ", ms_icp_time)
    #print("Inlier Fitness: ", registration_ms_icp.fitness)
    #print("Inlier RMSE: ", registration_ms_icp.inlier_rmse)
    
    #master = master.transform(registration_ms_icp.transformation) + target_cuda 
    #hotpink = hotpink.transform(registration_ms_icp.transformation) + o3d.t.geometry.PointCloud().from_legacy(maskpcds[i]) 
        
    
print(time.time() - tottime)
master = master.to_legacy()
#hotpink = hotpink.to_legacy()
o3d.visualization.draw_geometries([master])
#o3d.visualization.draw_geometries([hotpink])

In [ ]:
o3d.visualization.draw_geometries([newpcd])

In [ ]:
def prepare_dataset(voxel_size):
    print(":: Load two point clouds and disturb initial pose.")

    demo_icp_pcds = o3d.data.DemoICPPointClouds()
    source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])
    target = o3d.io.read_point_cloud(demo_icp_pcds.paths[1])
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [ ]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 30
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    #pcd_down.orient_normals_consistent_tangent_plane(30)
    pcd_down.orient_normals_towards_camera_location()
    radius_feature = voxel_size * 2
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=30))
    print(pcd_fpfh)
    return pcd_down, pcd_fpfh

In [ ]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

In [ ]:
voxel_size = 0.2  # means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size)

In [ ]:
print(source)

In [ ]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    # result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    #     source_down, target_down, source_fpfh, target_fpfh, True,
    #     distance_threshold,
    #     o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
    #     3, [
    #         o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
    #         o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    #     ], o3d.pipelines.registration.RANSACConvergenceCriteria(1000000, 0.999))
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        2, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(1000000, 0.999))
    return result

In [ ]:
def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.pipelines.registration.FastGlobalRegistrationOption(maximum_correspondence_distance=distance_threshold))
    return result

In [ ]:
o3d.visualization.draw_geometries([pcds[0], pcds[100]])

In [ ]:

voxel_size = .02
pcd_down_s, pcd_fpfh_s = preprocess_point_cloud(copy.deepcopy(pcds[0]), voxel_size)    
pcd_down_t, pcd_fpfh_t = preprocess_point_cloud(copy.deepcopy(pcds[100]), voxel_size)
    
o3d.visualization.draw_geometries([pcd_down_s, pcd_down_t])
    
result_ransac = execute_global_registration(pcd_down_s, pcd_down_t , pcd_fpfh_s, pcd_fpfh_t, voxel_size)
#
#result_ransac = execute_fast_global_registration(pcd_down_s, pcd_down_t,
#                                            pcd_fpfh_s, pcd_fpfh_t,
#                                            voxel_size)

print(result_ransac)
o3d.visualization.draw_geometries([copy.deepcopy(pcds[0]).transform(result_ransac.transformation), pcds[100]])


In [ ]:
o3d.visualization.draw_geometries(newpcd[:100])

In [ ]:
o3d.visualization.draw_geometries(masters)

In [ ]:
pcds = newpcd 
tottime = time.time()
#master =  o3d.t.geometry.PointCloud().from_legacy(pcds[0])
master = None
#hotpink =  o3d.t.geometry.PointCloud().from_legacy(maskpcds[0])
#master.estimate_normals(max_nn=30, radius=.05)

# vis.clear_geometries()
# vis.add_geometry(master.to_legacy())

#updatevis(vis, master.to_legacy())
transforms =[]
masters = []      
for i in range(0, 100):    
    s = time.time()   
    
    
    #source_cuda = master
    source_cuda = o3d.t.geometry.PointCloud().from_legacy(pcds[i-1])
    #source_cuda = source_cuda.cuda(0) 
    
    target_cuda = o3d.t.geometry.PointCloud().from_legacy(pcds[i])
    #target_cuda = target_cuda.cuda(0)  
    
    
#     pcd_down_s, pcd_fpfh_s = preprocess_point_cloud(source_cuda, 0.01)
    
#     pcd_down_t, pcd_fpfh_t = preprocess_point_cloud(target_cuda, 0.01)
    
#     result_ransac = execute_fast_global_registration(pcd_down_s, pcd_down_t , pcd_fpfh_s, pcd_fpfh_t, 0.01)
    
    
    
    criteria = o3d.t.pipelines.registration.ICPConvergenceCriteria(relative_fitness=0.000001,
                                       relative_rmse=0.000001,
                                       max_iteration=1000)
    
    registration_ms_icp = o3d.t.pipelines.registration.icp(source_cuda, target_cuda, .1,
                             estimation_method =estimation, criteria=criteria, voxel_size=0.01)
    
    
    
#     registration_ms_icp = o3d.t.pipelines.registration.multi_scale_icp(master, target_cuda,
#                                            voxel_sizes, criteria_list,
#                                            max_correspondence_distances, estimation_method = estimation)#, init_source_to_target = result_ransac.transformation)#, save_loss_log=False)#,
    #print(  result_ransac.transformation)                                     

    #source_cuda = source_cuda.cpu()
    #target_cuda = target_cuda.cpu()
    ms_icp_time = time.time() - s
    
    
    # if registration_ms_icp.fitness < .99:
    #     s = source_cuda.to_legacy()
    #     s.paint_uniform_color([1.0, 0, 0])
    #     t = target_cuda.to_legacy()
    #     t.paint_uniform_color([0, 0, 1.0])
    #     vis.add_geometry(t, False)
        
    
    print("Time taken by Multi-Scale ICP: ", ms_icp_time)
    print("Inlier Fitness: ", registration_ms_icp.fitness)
    print("Inlier RMSE: ", registration_ms_icp.inlier_rmse)
    
    if master == None:
        master = target_cuda
    else:
        transforms.append(result_ransac.transformation)
        master = master.transform(result_ransac.transformation)
        master = master + target_cuda
        #master = master + target_cuda
        #master = master.transform(registration_ms_icp.transformation) + target_cuda
    masters.append(copy.deepcopy(source_cuda.transform(result_ransac.transformation).to_legacy()))
    #master = master.voxel_down_sample(voxel_size=0.01)
    #hotpink = hotpink.transform(registration_ms_icp.transformation) + o3d.t.geometry.PointCloud().from_legacy(maskpcds[i]) 
    #time.sleep(10)
        




In [ ]:
print(len(masters))

In [ ]:
print(len(transforms))

In [ ]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(masters[0])

s = time.time()
i = 1
while not keyboard.is_pressed('esc'):    
    if time.time() - s > .5 and i < len(masters):
        vis.add_geometry(masters[i], False)
        #vis.update_geometry(pcds[0].transform(transforms[i]), False)
        i = i + 1
        s = time.time()
    
    vis.poll_events()
    vis.update_renderer()  
vis.destroy_window()

In [ ]:
pcd = copy.deepcopy(hotpink)
pcd = pcd.voxel_down_sample(voxel_size=0.025)
plane_model, inliers = pcd.segment_plane(distance_threshold=0.2,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([outlier_cloud])
#o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [ ]:


plane_model, inliers = outlier_cloud.segment_plane(distance_threshold=0.08,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = outlier_cloud.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = outlier_cloud.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([outlier_cloud])
#o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [ ]:

pinkcolors = np.asarray(outlier_cloud.colors)
pinkpoints = np.asarray(outlier_cloud.points)
pcolors = pinkcolors * 255
pink = np.where(pcolors[:,0] == 255)
hotpoints = pinkpoints[pink]
hotcolors = pinkcolors[pink]
hotpink2 = o3d.geometry.PointCloud()        
hotpink2.points = o3d.utility.Vector3dVector(hotpoints)    
hotpink2.colors = o3d.utility.Vector3dVector(hotcolors)    
o3d.visualization.draw_geometries([hotpink2])

In [ ]:
hotpink2, o = hotpink2.remove_radius_outlier(15, .05,True)
o3d.visualization.draw_geometries([hotpink2])

In [ ]:

pcd = outlier_cloud.voxel_down_sample(voxel_size=0.025)
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd.cluster_dbscan(eps=0.1, min_points=30, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.visualization.draw_geometries([pcd])

In [ ]:
print("Full registration ...")
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Info) as cm:
    pose_graph = full_registration(pcds[:10])

In [ ]:
print("Optimizing PoseGraph ...")
option = o3d.pipelines.registration.GlobalOptimizationOption(
    max_correspondence_distance=max_correspondence_distances[2],
    edge_prune_threshold=0.25,
    reference_node=0)
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    o3d.pipelines.registration.global_optimization(
        pose_graph,
        o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
        o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
        option)

In [ ]:
print("Transform points and display")
legacy = []
for point_id in range(len(pcds[:50])):
    print(pose_graph.nodes[point_id].pose)
    p = copy.deepcopy(pcds[point_id])
    p.transform(pose_graph.nodes[point_id].pose)
    legacy.append(p)
o3d.visualization.draw_geometries(legacy)



In [ ]:
#pcds = load_point_clouds(voxel_size)
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(pcds)):
    pcds[point_id].transform(pose_graph.nodes[point_id].pose)
    pcd_combined += pcds[point_id]
pcd_combined_down = pcd_combined.voxel_down_sample(voxel_size=voxel_size*.5)
#o3d.io.write_point_cloud("multiway_registration.pcd", pcd_combined_down)
o3d.visualization.draw_geometries([pcd_combined_down],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

In [ ]:
master = None
for _ in range(5):
    voxel_size = 0.05
    pcds_down, pcds = load_point_clouds(voxel_size)
    #o3d.visualization.draw_geometries(pcds_down)
    print("Full registration ...")
  
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Info) as cm:
        pose_graph = full_registration(pcds_down,
                                   max_correspondence_distance_coarse,
                                   max_correspondence_distance_fine)
    
    print("Transform points and display")
    for point_id in range(len(pcds_down)):
        #print(pose_graph.nodes[point_id].pose)
        pcds_down[point_id].transform(pose_graph.nodes[point_id].pose)
    #o3d.visualization.draw_geometries(pcds_down)
    
    
    pcd_combined = o3d.geometry.PointCloud()
    for point_id in range(len(pcds)):
        pcds[point_id].transform(pose_graph.nodes[point_id].pose)
        pcd_combined += pcds[point_id]
    pcd_combined_down = pcd_combined.voxel_down_sample(voxel_size=voxel_size*.5)
    
    if master == None:
        master = pcd_combined_down
    else:
        
        source_down, source_fpfh = preprocess_point_cloud(master, voxel_size)
        target_down, target_fpfh = preprocess_point_cloud(pcd_combined_down, voxel_size)

        start = time.time()
        
        result_ransac = execute_fast_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
    
        print(result_ransac)
        print("Global registration took %.3f sec.\n" % (time.time() - start))
    
    
    
        master.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
        pcd_combined_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    
        start = time.time()
        result_icp = refine_registration(master, pcd_combined_down, source_fpfh, target_fpfh, voxel_size)
        
    
        master = master.transform(result_icp.transformation)
        master = master + pcd_combined_down
        master = master.voxel_down_sample(voxel_size=voxel_size*.5)
        print(result_icp)
        print("Point to Plane registration took %.3f sec.\n" % (time.time() - start))
        # radius_normal = voxel_size * 2
        # pcd_combined_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
        # transformation_icp, information_icp = pairwise_registration(master, pcd_combined_down)
        # pcd_combined_down = pcd_combined_down.transform(transformation_icp)
        # master = master + pcd_combined_down
    
    
    
o3d.visualization.draw_geometries([master])


In [ ]:
# Input point-clouds
demo_icp_pcds = o3d.data.DemoICPPointClouds()
source = o3d.t.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.t.io.read_point_cloud(demo_icp_pcds.paths[1])

source_cuda = source.cuda(0)
target_cuda = target.cuda(0)
s = time.time()

registration_ms_icp = treg.multi_scale_icp(source_cuda, target_cuda,
                                           voxel_sizes, criteria_list,
                                           max_correspondence_distances,
                                           init_source_to_target, estimation,
                                           save_loss_log)

ms_icp_time = time.time() - s
print("Time taken by Multi-Scale ICP: ", ms_icp_time)
print("Inlier Fitness: ", registration_ms_icp.fitness)
print("Inlier RMSE: ", registration_ms_icp.inlier_rmse)

draw_registration_result(source.cpu(), target.cpu(),
                         registration_ms_icp.transformation)

In [ ]:
o3d.visualization.draw_geometries([master])

In [ ]:

pcd = pcd_combined_down

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        pcd.cluster_dbscan(eps=0.08, min_points=50, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.455,
                                  front=[-0.4999, -0.1659, -0.8499],
                                  lookat=[2.1813, 2.0619, 2.0999],
                                  up=[0.1204, -0.9852, 0.1215])

In [ ]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target],
                                      zoom=0.5,
                                      front=[-0.2458, -0.8088, 0.5342],
                                      lookat=[1.7745, 2.2305, 0.9787],
                                      up=[0.3109, -0.5878, -0.7468])

In [ ]:
print("1. Load two point clouds and show initial pose")
demo_colored_icp_pcds = o3d.data.DemoColoredICPPointClouds()
source = o3d.io.read_point_cloud(demo_colored_icp_pcds.paths[0])
target = o3d.io.read_point_cloud(demo_colored_icp_pcds.paths[1])

# draw initial alignment
#current_transformation = np.identity(4)
#draw_registration_result_original_color(source, target, current_transformation)

In [ ]:
print(source.points[0])